In [1]:
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import math

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 50)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_seq_items', 50)
pd.set_option('display.width', 100)

In [3]:
#chargement des base de données
df_geolocation = pd.read_csv('Data/olist_geolocation_dataset.csv')
df_orders = pd.read_csv('Data/olist_orders_dataset.csv')
df_order_items = pd.read_csv('Data/olist_order_items_dataset.csv')
df_order_reviews = pd.read_csv('Data/olist_order_reviews_dataset.csv')
df_customers = pd.read_csv('Data/olist_customers_dataset.csv')
df_order_payments = pd.read_csv('Data/olist_order_payments_dataset.csv')
df_products = pd.read_csv('Data/olist_products_dataset.csv')
df_sellers = pd.read_csv('Data/olist_sellers_dataset.csv')
df_product_category_name = pd.read_csv('Data/product_category_name_translation.csv')

## Fonctions

In [4]:
def deg2rad(x):
    return math.pi*x/180

def get_distance_m(lat1, lng1, lat2, lng2):
    earth_radius = 6378137   # Terre = sphère de 6378km de rayon
    rlo1 = deg2rad(lng1)    # CONVERSION
    rla1 = deg2rad(lat1)
    rlo2 = deg2rad(lng2)
    rla2 = deg2rad(lat2)
    dlo = (rlo2 - rlo1) / 2
    dla = (rla2 - rla1) / 2
    a = (np.sin(dla) * np.sin(dla)) + np.cos(rla1) * np.cos(rla2) * (np.sin(dlo) * np.sin(dlo))
    d = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return (earth_radius * d)

## Nettoyage de base

In [5]:
# prepare customers
df_geolocation = df_geolocation.groupby(['geolocation_zip_code_prefix']).mean()
df_customers = df_customers.merge(df_geolocation, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix')

# prépare order
df_orders = df_orders.merge(df_customers[['customer_unique_id', 'customer_id']], on='customer_id')
df_orders = df_orders.merge(df_order_reviews, on='order_id', how='left')
# keep only last review if multiple
df_orders = df_orders.sort_values(by='review_answer_timestamp').drop_duplicates(subset=['order_id'], keep='last')

for i in df_order_payments.index:
    df_order_payments.loc[i,df_order_payments.loc[i,'payment_type']] = df_order_payments.loc[i,'payment_value']
df_order_payments = df_order_payments.join(pd.get_dummies(df_order_payments['payment_type']), rsuffix="_bin")
df_order_payments.fillna(0, inplace=True)
df_payment = pd.DataFrame({'order_id' : df_order_payments['order_id'].value_counts().index, 'number_of_payment' : df_order_payments['order_id'].value_counts()})
df_payment = df_payment.merge(df_order_payments[['order_id', 'payment_value', 'boleto', 'credit_card', 'debit_card', 'not_defined', 'voucher', 'boleto_bin', 'credit_card_bin', 'debit_card_bin', 'not_defined_bin', 'voucher_bin']].groupby('order_id').sum(), on='order_id')
df_orders = df_orders.merge(df_payment, on='order_id')

# prépare order items
df_sellers = df_sellers.merge(df_geolocation, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
df_products = df_products.merge(df_product_category_name, on='product_category_name', how='left')
df_order_items = df_order_items.merge(df_sellers, on='seller_id', how='left')
df_order_items = df_order_items.merge(df_products, on='product_id', how='left')
df_order_items = df_order_items.merge(df_orders[['order_id', 'customer_unique_id']], on='order_id')

In [6]:
# prosessing on df_customers
df_customers['customer_city'] = df_customers['customer_city'].str.title()

# prosessing on df_orders
date_columns = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'review_creation_date', 'review_answer_timestamp']
for col in date_columns:
    df_orders[col] = pd.to_datetime(df_orders[col], format='%Y-%m-%d %H:%M:%S')


df_orders['order_status'] = df_orders['order_status'].str.title()
# Engineering new/essential columns
df_orders['order_purchase_year'] = df_orders.order_purchase_timestamp.apply(lambda x: x.year)
df_orders['order_purchase_month'] = df_orders.order_purchase_timestamp.apply(lambda x: x.month)
df_orders['order_purchase_dayofweek'] = df_orders.order_purchase_timestamp.apply(lambda x: x.dayofweek)
df_orders['order_purchase_hour'] = df_orders.order_purchase_timestamp.apply(lambda x: x.hour)
df_orders['order_purchase_day'] = df_orders['order_purchase_dayofweek'].map({0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'})
df_orders['order_purchase_mon'] = df_orders.order_purchase_timestamp.apply(lambda x: x.month).map({1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'})


In [7]:
# Extracting attributes for purchase date - Time of the Day
hours_bins = [-0.1, 6, 12, 18, 23]
hours_labels = ['Dawn', 'Morning', 'Afternoon', 'Night']
df_orders['order_purchase_time_day'] = pd.cut(df_orders['order_purchase_hour'], hours_bins, labels=hours_labels)

# Changing the month attribute for correct ordenation
df_orders['month_year'] = df_orders['order_purchase_month'].astype(str).apply(lambda x: '0' + x if len(x) == 1 else x)
df_orders['month_year'] = df_orders['order_purchase_year'].astype(str) + '-' + df_orders['month_year'].astype(str)
# Creating year month column
df_orders['month_y'] = df_orders['order_purchase_timestamp'].map(lambda date: 100*date.year + date.month)

# prosessing on df_order_items
df_order_items['seller_city'] = df_order_items['seller_city'].str.title()
df_order_items['product_category_name'] = df_order_items['product_category_name'].str.replace('_', ' ').str.title()
df_order_items['product_category_name_english'] = df_order_items['product_category_name_english'].str.replace('_', ' ').str.title()
df_order_items['nbItems'] = 1

df_order_items = df_order_items.merge(df_customers[['customer_unique_id', 'geolocation_lat', 'geolocation_lng']], on="customer_unique_id", suffixes=('_seller', '_customer'))
df_order_items['euclidean_distance'] = math.pi*np.sqrt(((df_order_items['geolocation_lat_seller']-df_order_items['geolocation_lat_customer'])**2+(df_order_items['geolocation_lng_seller']-df_order_items['geolocation_lng_customer'])**2))/180 * 6378137
df_order_items['orthodromique_distance'] = get_distance_m(df_order_items['geolocation_lat_seller'], df_order_items['geolocation_lng_seller'], df_order_items['geolocation_lat_customer'], df_order_items['geolocation_lng_customer'])
df_order_items['orthodromique_distance_log'] = np.log(df_order_items['orthodromique_distance']+1)
df_order_items['product_volume'] = df_order_items['product_height_cm'] * df_order_items['product_length_cm'] * df_order_items['product_width_cm']
df_order_items[['product_volume_log', 'product_weight_g_log']] = np.log(df_order_items[['product_volume', 'product_weight_g']]+1)


In [8]:
# relative time
df_orders['delivery_against_estimated'] = (df_orders['order_estimated_delivery_date'] - df_orders['order_delivered_customer_date']).astype('timedelta64[h]')/24
df_orders['order_approved_timelapse'] = (df_orders['order_approved_at'] - df_orders['order_purchase_timestamp']).astype('timedelta64[h]')
df_orders['order_delivered_carrier_timelapse'] = (df_orders['order_delivered_carrier_date'] - df_orders['order_purchase_timestamp']).astype('timedelta64[h]')/24
df_orders['order_delivered_customer_timelapse'] = (df_orders['order_delivered_customer_date'] - df_orders['order_purchase_timestamp']).astype('timedelta64[h]')/24
df_orders['order_estimated_delivery_timelapse'] = (df_orders['order_estimated_delivery_date'] - df_orders['order_purchase_timestamp']).astype('timedelta64[h]')/24
df_orders['review_creation_timelapse_from_delivery'] = (df_orders['review_creation_date'] - df_orders['order_delivered_customer_date']).astype('timedelta64[m]')
df_orders['review_answer_timeslapse_from_review'] = (df_orders['review_answer_timestamp'] - df_orders['review_creation_date']).astype('timedelta64[h]')

df_orders.drop(['review_comment_title', 'review_comment_message'], axis=1, inplace=True)
df_order_items['product_category_name_english'].fillna('unknown', inplace=True)

df_orders = df_orders[df_orders['order_status'] == 'Delivered']
df_orders.drop(['order_status'], axis=1, inplace=True)
df_orders.dropna(inplace=True)

df_onehot = pd.get_dummies(df_orders[['order_purchase_day', 'order_purchase_mon', 'order_purchase_time_day']], prefix=['order_purchase_day', 'order_purchase_mon', 'order_purchase_time_day'])
df_orders = df_orders.join(df_onehot)

In [21]:
df_orders['order_purchase_timestamp'].max() - pd.to_timedelta(3, unit='d')

Timestamp('2018-08-26 15:00:37')

In [17]:
pd.to_timedelta(3, unit='d')
Préparation des différents Datasets


Timedelta('3 days 00:00:00')

## Final DataFrame

In [9]:
df = pd.DataFrame({'customer_unique_id' : df_customers['customer_unique_id'].value_counts().index})
df = df.merge(df_customers.groupby('customer_unique_id').first()[['customer_city']], on='customer_unique_id')

city_count = df['customer_city'].value_counts()
for i in range(df.shape[0]):
    val = df.index[i]
    df.loc[val, 'customer_city_size']= city_count[df.loc[val, 'customer_city']]
    
# Merge df_orders
order_agg = {
            'order_purchase_timestamp': 'max',
            'payment_value': 'sum'
            }
df = df.merge(df_orders.groupby('customer_unique_id').agg(order_agg), on='customer_unique_id')

# Merge df_order_items
order_items_agg = {'price': 'mean',
                   'freight_value': 'sum',
                   'orthodromique_distance_log': 'mean',
                   'product_volume_log': 'mean'
                  }
df = df.merge(df_order_items.groupby('customer_unique_id').agg(order_items_agg), on='customer_unique_id')

# fillna
fill_dic = {
            'orthodromique_distance_log': df['orthodromique_distance_log'].mean(),
            'product_volume_log': df['product_volume_log'].mean()
           }
df.fillna(fill_dic, inplace=True)

# rename columns
df.rename(columns={
                   'order_purchase_timestamp': 'recency',
                   'payment_value': 'revenue',
                   'price': 'mean_price',
                   'freight_value': 'mean_freight_value'
                  }, inplace=True)
df['recency'] = (df['recency'].max() - df['recency']).astype('timedelta64[h]')/24
df.dropna(inplace=True)